In [1]:
!apt-get update
!apt-get install -y tesseract-ocr tesseract-ocr-ben poppler-utils
!pip install pytesseract pdf2image pillow pandas regex


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,287 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,966 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,634 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 

In [ ]:
from google.colab import files
uploaded = files.upload()
pdf_name = list(uploaded.keys())[0]
print("PDF:", pdf_name)


In [ ]:
import subprocess

def get_page_count(pdf):
    result = subprocess.run(
        ["pdfinfo", pdf],
        stdout=subprocess.PIPE,
        text=True
    )
    for line in result.stdout.split("\n"):
        if "Pages:" in line:
            return int(line.split(":")[1].strip())

total_pages = get_page_count(pdf_name)
print("Total pages:", total_pages)


In [ ]:
from pdf2image import convert_from_path
import pytesseract
import pandas as pd
import re
import gc

def clean_bangla(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\u0980-\u09FF0-9।, ]', '', text)
    return text.strip()

rows = []
BATCH_SIZE = 5       # 🔥 KEY: small batch
DPI = 200            # 🔥 lower DPI = less memory

for start in range(1, total_pages + 1, BATCH_SIZE):
    end = min(start + BATCH_SIZE - 1, total_pages)
    print(f"OCR pages {start}–{end}")

    pages = convert_from_path(
        pdf_name,
        dpi=DPI,
        first_page=start,
        last_page=end
    )

    for i, page in enumerate(pages):
        text = pytesseract.image_to_string(page, lang="ben")
        text = clean_bangla(text)

        if len(text) > 30:
            rows.append({
                "file": pdf_name,
                "page": start + i,
                "text": text
            })

    del pages
    gc.collect()   # 🔥 FREE MEMORY


In [ ]:
df = pd.DataFrame(rows)

csv_name = "ocr_extracted_text.csv"
df.to_csv(csv_name, index=False, encoding="utf-8-sig")

print("✅ CSV saved:", csv_name)
print("Total pages extracted:", len(df))
df.head()
